# TEMPLATE: Build evaluation

In [7]:
import yaml
from collections import Counter, defaultdict
import itertools
from math import comb
from tqdm.notebook import tqdm
import random
from functools import lru_cache

In [2]:
%load_ext autoreload
%autoreload 2

from lib.model import *
from lib.build_tools import *
from lib.loader_tools import *


In [3]:
IDEAS = load_ideas([
    'data/gypsy_transformed/ideas.yaml', 
    'data/gypsy_transformed/ideas_flogi.yaml', 
    'data/gypsy_transformed/ideas_flogi_relig.yaml'
])

POLICIES = load_policies([
    'data/gypsy_transformed/policies.yaml'
])

In [4]:
def print_build(build: Build, effects: list[str]):
    print('--------------------------------------------------------------------')
    print(f"Build - score: {build.score} - ideas: {build.ideas}")
    for effect in effects:
        print(f"\t{effect}: {build.total_effect[effect]:.2f}")

def get_ideas_effect(ideas: tuple[str]):
    total_effect = Counter()
    for idea in ideas:
        total_effect.update(IDEAS[idea].effect)
        
    return total_effect

def compute_build(ideas: tuple[str], base_policy_slots=4, debug=False) -> Build:
    ideas_effect = get_ideas_effect(ideas)

    available_policies = get_available_policies(ideas, POLICIES)
    max_policy_slots = get_max_policy_slots(ideas_effect, base_policy_slots)

    adm_max_policy, dip_max_policy, mil_max_policy = get_max_policy_slots(ideas_effect, base_policy_slots)
    micro_management_policies_effect = get_micro_management_policies_effect(ideas, available_policies, max_policy_slots)
    war_policies_effect = get_war_policies_effect(ideas, MILITARY_WEIGHTS, available_policies, max_policy_slots)

    total_effect = Counter()
    total_effect.update(ideas_effect)
    total_effect.update(micro_management_policies_effect)
    total_effect.update(war_policies_effect)


    
    return Build(
        ideas=ideas, 
        score=score(total_effect, [COUNTRY_WEIGHTS, MILITARY_WEIGHTS], debug=debug),
        total_effect=total_effect,
        war_policies_effect=war_policies_effect,
    )

## Build score

In [5]:
# Country weights are used to determine the score of a country during peace time
COUNTRY_WEIGHTS = {
    'estate_nationalist': 10,               # Having this estate is 10 point
    'development_cost': 150,               # Each 10% is 15 points
    'free_policy': 5,                       # Each free extra policy is 5 points
    'free_adm_policy': 2,                   # Each free extra policy is 2 points
    'free_dip_policy': 2,                   # Each free extra policy is 2 points
    'free_mil_policy': 1,                   # Each free extra policy is 1 points
    'possible_policy': 12,                  # Each extra policy column is 12 points
    'possible_adm_policy': 4,               # Each possible policy is 4 points
    'possible_dip_policy':  4,              # Each possible policy is 4 points
    'possible_mil_policy': 4,               # Each possible policy is 4 points
    'technology_cost': 60,                 # Each 10% discount is 6 points
    'adm_tech_cost_modifier': 20,          # Each 10% discount is 2 point
    'dip_tech_cost_modifier':  20,         # Each 10% discount is 2 point
    'mil_tech_cost_modifier':  20,         # Each 10% discount is 2 point
    'idea_cost': 60,                       # Each 10% discount is 6 points
    'governing_capacity_modifier': 30,      # Each 10% is 3 points
    'advisor_cost': 10,                    # Each 10% discount is 1 point
    'global_tax_modifier': 5,               # Each 10% is 0.5 point
    'production_efficiency': 5,             # Each 10% is 0.5 point
    'global_trade_goods_size_modifier': 5,  # Each 10% is 0.5 point
    'trade_efficiency': 5,                  # Each 10% is 0.5 point
    'state_maintenance_modifier': 5,       # Each 10% discount is 0.5 point
}

# War weights are used to determine the score of a country during war time and to find the best policies
MILITARY_WEIGHTS ={
    'infantry_power': 150,                  # Each 10% is 15 points
    'cavalry_power': 150,                   # Each 10% is 15 points
    'artillery_power': 150,                 # Each 10% is 15 points
    'discipline': 300,                      # Each 10% is 30 points
    'fire_damage': 150,                     # Each 10% is 15 points
    'fire_damage_received': 150,           # Each 10% is 15 points
    'shock_damage': 150,                    # Each 10% is 15 points
    'shock_damage_received': 150,          # Each 10% is 15 points
    'land_morale': 150,                     # Each 10% is 15 points
    'global_manpower_modifier': 50,         # Each 10% is 5 points
    'land_forcelimit_modifier': 50,         # Each 10% is 5 points
    'land_maintenance_modifier': 20        # Each 10% discount is 2 points
}

## Build eval

In [9]:
build = compute_build(ideas=(
    'decentralism', 
    'fem_boy', 
    'development', 
    'trade_ideas', 
    'imperialismus', 
    'standing_army', 
    'quality', 
    'weapon_quality'
))
print_build(build, effects=MILITARY_WEIGHTS.keys())
print_build(build, effects=COUNTRY_WEIGHTS.keys())

--------------------------------------------------------------------
Build - score: 434.15000000000003 - ideas: ('decentralism', 'fem_boy', 'development', 'trade_ideas', 'imperialismus', 'standing_army', 'quality', 'weapon_quality')
	infantry_power: 0.30
	cavalry_power: 0.30
	artillery_power: 0.30
	discipline: 0.28
	fire_damage: 0.00
	fire_damage_received: 0.00
	shock_damage: 0.00
	shock_damage_received: 0.00
	land_morale: 0.45
	global_manpower_modifier: 0.30
	land_forcelimit_modifier: 0.20
	land_maintenance_modifier: 0.00
--------------------------------------------------------------------
Build - score: 434.15000000000003 - ideas: ('decentralism', 'fem_boy', 'development', 'trade_ideas', 'imperialismus', 'standing_army', 'quality', 'weapon_quality')
	estate_nationalist: 0.00
	development_cost: 0.60
	free_policy: 0.00
	free_adm_policy: 0.00
	free_dip_policy: 0.00
	free_mil_policy: 0.00
	possible_policy: 0.00
	possible_adm_policy: 0.00
	possible_dip_policy: 0.00
	possible_mil_policy: 0

## Allowed idea values

### Admin

- innovativeness_ideas
- economic_ideas
- expansion_ideas
- administrative_ideas
- humanist_ideas
- judiciary
- development
- strong_men
- fem_boy 
- public_admin
- centralism
- decentralism

#### Gov type

- monarchie0
- republik0
- aristo0
- diktatur0
- horde0

#### Religion

- religious_ideas
- katholisch0
- protestant0
- reformiert0
- orthodox0
- islam0
- tengri0
- hindu0
- confuci0
- budda0
- norse0
- shinto0
- cathar0
- coptic0
- romuva0
- suomi0
- jew0
- slav0
- helle0
- mane0
- animist0
- feti0
- zoro0
- ancli0
- nahu0
- mesoam0
- inti0
- tote0
- shia0
- ibadi0
- hussite0
- alche0

### Diplo

- spy_ideas
- dynastic
- influence_ideas
- trade_ideas
- exploration_ideas
- maritime_ideas
- heavy_ship
- galley_ship
- trade_ship
- colonial_emp
- assimilation
- sociaty
- propaganda
- fleet_base
- nationalismus
- konigreich0
- imperialismus

### Military

- offensive
- defensive
- quality
- quantity
- general_staff
- standing_army
- conscription
- merc_army
- weapon_quality
- fortress
- war_production
- formation0
- militarism
- shock_ideas
- fire_ideas